In [1]:
import sys 
sys.path.append("../")
from LIB.utils import * 

In [2]:
train=pd.read_json("./train.zip")
test=pd.read_json("./test_questions.zip")

In [3]:
json_columns=[]
for col in train.columns : 
    try :
         train[col].unique()
    except : 
        json_columns.append(col) 

In [4]:
col_to_remove=["contributors","retweeted","coordinates","extended_entities","geo","place","quoted_status","retweeted_status"]
train.drop(col_to_remove,axis=1,inplace=True)
test.drop(col_to_remove,axis=1,inplace=True)

json_columns=[]
for col in train.columns : 
    try :
         train[col].unique()
    except : 
        json_columns.append(col)  
def parse_json(x):
    for col in json_columns :
        print(col)
        for key  in x[col].iloc[0].keys():
            x[col+"_"+key]=x[col].apply(lambda x: x[key])
    return x 
train=parse_json(train)
test=parse_json(test)
json_columns=[]
for col in train.columns : 
    try :
         a=train[col].unique()
    except : 
        json_columns.append(col)

entities
user
entities
user


In [5]:
cols=["entities_symbols","entities_hashtags","entities_urls"]
for col in cols  : 
    train[col]=train[col].apply(lambda x : x if len(x)>0 else  np.nan)
    test[col]=test[col].apply(lambda x : x if len(x)>0 else  np.nan)    

In [6]:
col_to_remove=["entities","id_str","in_reply_to_screen_name","in_reply_to_status_id_str","in_reply_to_user_id_str",
              "quoted_status_id_str","text","user","entities_symbols","user_time_zone","user_time_zone",
              "user_profile_sidebar_fill_color","user_profile_banner_url","user_is_translator",
              "user_has_extended_profile","user_translator_type","user_profile_link_color","user_screen_name",                                                        
               "user_profile_link_color","user_profile_background_image_url",
               "user_utc_offset","user_protected","user_profile_background_color",
              "user_geo_enabled","user_profile_image_url","user_profile_use_background_image","user_description",
               "user_profile_image_url_https","user_profile_background_tile","user_following","user_contributors_enabled",
               "user_id_str","user_name","user_profile_background_image_url_https","user_profile_sidebar_border_color",
               "user_default_profile_image","user_url","user_notifications","user_profile_text_color" ,"user_lang",
              "quoted_status_id","user_follow_request_sent","favorited"]

train["len_text"]=train.text.apply(lambda x :len(x.split(" ")))
test["len_text"]=train.text.apply(lambda x :len(x.split(" ")))

train.drop(col_to_remove,axis=1,inplace=True)
test.drop(col_to_remove,axis=1,inplace=True)

In [7]:
json_columns=[]
for col in train.columns : 
    try :
         a=train[col].unique()
    except : 
        json_columns.append(col) 

In [8]:
def time_features(x):
    x["created_at"]=pd.to_datetime(x["created_at"])
    x["year"]=x["created_at"].dt.year
    x["month"]=x["created_at"].dt.month       
    x["dayofweek"]=x["created_at"].dt.dayofweek
    x["hour"]=x["created_at"].dt.hour
    return x 
train=time_features(train)
test=time_features(test)


In [9]:
col_to_factorize=["year","user_location","user_id","user_default_profile","user_verified",
                 "truncated", "source" ,"lang","is_quote_status","user_is_translation_enabled"]
map_categorical_feautres(Data=[train,test],columns=col_to_factorize)

year
user_location
user_id
user_default_profile
user_verified
truncated
source
lang
is_quote_status
user_is_translation_enabled


In [10]:
train["is_reply_to_status_id"]=train["in_reply_to_status_id"].apply(lambda x: 1  if  np.isnan(x)  else  0 ) 
test["is_reply_to_status_id"]=test["in_reply_to_status_id"].apply(lambda x: 1  if  np.isnan(x)  else  0 ) 
train["is_reply_to_user_id"]=train["in_reply_to_user_id"].apply(lambda x: 1 if np.isnan(x)  else  0 ) 
test["is_reply_to_user_id"]=test["in_reply_to_user_id"].apply(lambda x: 1 if np.isnan(x)  else  0 ) 
train["reply_to_user_id_count"]=train.groupby("in_reply_to_user_id").in_reply_to_user_id.transform("count").fillna(0)
test["reply_to_user_id_count"]=test.groupby("in_reply_to_user_id").in_reply_to_user_id.transform("count").fillna(0)

train["reply_to_status_id_count"]=train.groupby("in_reply_to_status_id").in_reply_to_user_id.transform("count").fillna(0)
test["reply_to_status_id_count"]=test.groupby("in_reply_to_status_id").in_reply_to_user_id.transform("count").fillna(0)

In [11]:
train["count_entities_user_mentions"]=train["entities_user_mentions"].apply(len)
test["count_entities_user_mentions"]=test["entities_user_mentions"].apply(len)

train["count_entities_hashtags"]=train["entities_hashtags"].fillna("").apply(len)
test["count_entities_hashtags"]=test["entities_hashtags"].fillna("").apply(len)

train["count_entities_urls"]=train["entities_urls"].fillna("").apply(len)
test["count_entities_urls"]=test["entities_urls"].fillna("").apply(len)

In [14]:
TF_idf=pd.read_csv("../data_proc/TF_IDF_20.csv")
# TF_idf=pd.read_csv("./TF_IDF_50_new.csv")


train=train.merge(TF_idf,on="id",how="left")
test=test.merge(TF_idf,on="id",how="left")


In [15]:
v="_0"
train.to_csv("../data_proc/train{}.csv".format(v),index=False)
test.to_csv("../data_proc/test{}.csv".format(v),index=False)

In [16]:
rm -r ../LIB/AE/